# Tutorial 4. Example of Training an Image Classifier
<br> This tutorial demostrates an example of training an image classifier with Pytorch and then transfer the training model to GPU. 

## 1. Data description
<br>Generally, when you have to deal with image, text, audio or video data, you can use standard python packages that load data into a numpy array. Then you can convert this array into a ``torch.*Tensor``. 

<br> Here is a list of commonly used python packages to deal with different types of data:

- For images, packages such as Pillow, OpenCV are useful
- For audio, packages such as scipy and librosa
- For text, either raw Python or Cython based loading, or NLTK and SpaCy are useful

<br> Specially, in pytorch, a new package ``torchvision`` is built to deal with image data, which has data loaders for common datasets such as Imagenet, CIFAR10, MNIST, etc. and data transformers for images, viz., ``torchvision.datasets`` and ``torch.utils.data.DataLoader``.

<br> In this tutorial, we will use the CIFAR10 dataset. It has the classes: ‘airplane’, ‘automobile’, ‘bird’, ‘cat’, ‘deer’, ‘dog’, ‘frog’, ‘horse’, ‘ship’, ‘truck’. The images in CIFAR-10 are of size 3x32x32, i.e. 3-channel color images of 32x32 pixels in size.


## 2. Training a classifier
